In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ./flaskwebapp/

/Users/shiwang/Desktop/Github/cloud_learn/flaskwebapp


In [3]:
#mnist_data = keras.datasets.mnist.load_data()
#pickle.dump(mnist_data, open( "mnist_data.p", "wb" ) )

# Copy model files

In [4]:
!cp ../../deep_learning_practice/model/model_mnist_cnn.h5 ./model/model_mnist_cnn.h5
!cp -r ../../nlp_practice/model/. ./model
!cp ../../nlp_practice/script/sentiment_utils.py ./script/tmp_sentiment_utils.py

# Scripts 

## Define utils

In [4]:
%%writefile ./script/imports.py
import numpy as np
import tensorflow as tf
import keras
import pickle
import spacy
import gensim
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
nlp = spacy.load('en')
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import corpora
bigram_model = Phraser.load('./model/phrase_model_bigram')
trigram_model = Phraser.load('./model/phrase_model_trigram')
dictionary = corpora.Dictionary.load('./model/dictionary')
lstm_model = keras.models.load_model('./model/yelp_lstm_sentiment.h5')
mnist_model = keras.models.load_model('./model/model_mnist_cnn.h5')

Overwriting ./script/imports.py


In [5]:
!cat ./script/imports.py ./script/tmp_sentiment_utils.py > ./script/sentiment_utils.py

## Define  `serve.py`

In [13]:
%%writefile serve.py

## sentiment
from script.sentiment_utils import *

def serve_model_sentiment(x):
    X = apply_phrase_model([x])
    X = np.array([dictionary.doc2idx(pad_trim_review(x), unknown_word_index=len(dictionary)) for x in X])
    prediction = lstm_model.predict(X)
    return str(prediction[0][0])

## mnist
def get_mnist_data():
    mnist_data = pickle.load( open( "mnist_data.p", "rb" ) )
    (X_train, y_train), (X_test, y_test) = mnist_data 
    X = X_test.reshape(X_test.shape[0], 28, 28, 1)
    X = X.astype('float32')
    X /= 255
    return X_test, X

def plot_4_digits(imgs):
    plt.figure(figsize=(2, 2))
    for i in range(4):
        img = imgs[i]
        plt.subplot(2, 2, i+1)
        plt.imshow(img, cmap='gray')
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    return plt

def serve_model_mnist(random_index):
    raw_X, processed_X = get_mnist_data()
    raw_X, processed_X = raw_X[random_index], processed_X[random_index]
    plot = plot_4_digits(raw_X)
    prediction = np.argmax(mnist_model.predict(processed_X), axis=1)
    return plot, str(prediction)

Overwriting serve.py


In [7]:
from serve import *

Using TensorFlow backend.


## Define `app.py`

In [112]:
%%writefile app.py
import os
import sys
from flask import Flask, render_template, flash, abort,redirect, request,g,url_for, session, make_response, jsonify, send_file
from serve import *
import random

app = Flask(__name__)
app.secret_key = "super secret key"

@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

@app.route('/api/test')
def test():
    return jsonify(result = "ok")

@app.route('/api/test2')
def test2():
    raw_X, processed_X = get_4_samples()
    predictions = serve_model_mnist(processed_X)
    return jsonify(result = str(predictions))

@app.route('/')
@app.route('/index')
def begin():
    return render_template('base.html')

@app.route('/mnist')
def mnist():
    random_index = random.choices(range(10000),k=4)
    f_path = './static/images/' + '-'.join([str(x) for x in random_index]) + '.png'
    plot, prediction = serve_model_mnist(random_index)
    plot.savefig(f_path)
    flash('The predicted digits are ' + prediction)
    return render_template('mnist.html', url = f_path)

@app.route('/sentiment')
def sentiment():
    return render_template('sentiment.html')

@app.route('/submit_number', methods = ['POST'])
def submit_number():
    return redirect('/mnist')
    
@app.route('/submit_text', methods = ['POST'])
def submit_text():
    input_text = request.form['input_text']
    result = serve_model_sentiment(input_text)
    flash('The predicted star is ' + result)
    return redirect('/sentiment')

if __name__ == '__main__':
    app.run(host='0.0.0.0',port=80)

Overwriting app.py


## Define html files

In [187]:
%%writefile ./templates/base.html

<!DOCTYPE html>
<html>
<head>
	<title> Welcome to My API</title>
	<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<script type="text/javascript" src = "{{ url_for('static', filename = 'hello.js') }}"></script>
	<link rel=stylesheet type=text/css href="{{ url_for('static', filename='bootstrap-min.css')}}">
	<link rel=stylesheet type=text/css href="{{ url_for('static', filename='jumbotron-narrow.css')}}">
	<link rel=stylesheet type=text/css href="{{ url_for('static', filename='style.css')}}">
</head>


<body>
	<div class = "container">
		<div class="header">
			<nav>
				<ul class="nav nav-pills pull-right">
					<li role = "presentation" class = "active"><a href="/">Home</a>
				</ul>
			</nav>
			<h3 class="text-muted">ML applications</h3>
		</div>

        <div class="row marketing">
			<div class="col-lg-6">
				<h4> Image Classification</h4>
				<p>A CNN model to classify hand-written digits. The example is adapted from tutorials on tensorflow official site. Network includes convolutional layer, Maxpooling layer, Dropout layer and Dense layer. The model is implemented with Keras. </p>
			</div>
            
            <div class="col-lg-6">
				<h4> Sentiment Analysis</h4>
				<p>A LSTM model to predict the rating based on comment texts. The model is built based on some yelp review datasets. The NLP workflow includes preprocessing (tokenization lemmatization and stop word removal), phrase modeling and lstm model implemented with Keras. </p>
			</div>
		</div>
        

			<h3>Services</h3>
			<p class="lead"></p>
			<p><a class="btn btn-sm btn-success" href="{{ url_for('mnist') }}"
				   role="button"> Image Classification</a></p>
			<p><a class="btn btn-sm btn-success" href="{{ url_for('sentiment') }}"
				   role="button"> Sentiment Analysis</a></p>

        
        <div class="row marketing">
			<div class="col-lg-6">
                {% block body %}
                {% endblock %}
                
                {% for message in get_flashed_messages() %}
                    <div class=flash><font color="red">{{ message }}</font></div>
                {% endfor %}
			</div>
            
			<div class="col-lg-6">
				<h4>Contact</h4>
                <a href="https://github.com/shiwang0211">https://github.com/shiwang0211</a>
				<p>shiwang0211@gmail.com</p>
			</div>
		</div>

		<footer class="footer" >
			<p>&copy; Shi Wang 2018</p>
		</footer>


	</div>
</body>
</html>

Overwriting ./templates/base.html


In [188]:
%%writefile ./templates/mnist.html

{% extends "base.html" %}
{% block body %}

<p>You can click to change a set of random numbers</p>
    <form action="{{ url_for('submit_number') }}" method=post class=add-entry>
      <dl>
        <dd><input type=submit value=Change>
      </dl>
    </form>
    
<img src={{ url }} alt="Chart" height="150" width="150">

{% endblock %}

Overwriting ./templates/mnist.html


In [189]:
%%writefile ./templates/sentiment.html

{% extends "base.html" %}
{% block body %}

    <p>You can type in your comment text</p>
    <form action="{{ url_for('submit_text') }}" method=post class=add-entry>
      <dl>
        <dt>Input Text:
        <dd><input type=text size=30 name=input_text>
        <dd><input type=submit value=Submit>
      </dl>
    </form>

{% endblock %}

Overwriting ./templates/sentiment.html


# Local test

In [194]:
!docker build -q -t ml_api:latest .

sha256:287ea3b18a1fe29db3a29cc2bd067b047e51c97accaa6fc48395add15e191358


In [195]:
!docker stop `docker ps -qa`
!docker rm `docker ps -qa`

2769e015c4e0
2769e015c4e0


In [196]:
!docker run -d -it -p 5000:80 ml_api:latest

8cb7f8f5c071b40b7485a590c53cb4fd4457c26bad23f9c63983f1c93e9c192d


In [197]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED                  STATUS                  PORTS                  NAMES
8cb7f8f5c071        ml_api:latest       "python app.py"     Less than a second ago   Up Less than a second   0.0.0.0:5000->80/tcp   dreamy_albattani


- http://localhost:5000/api/test
- http://localhost:5000/index

# Deploy

see `Docker Learn.ipynb`